In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os

In [11]:
def score(reduc_time, accuracy, train_time, red_w = 200, ac_w = 1, train_w = 20):
    return (accuracy ** ac_w)*(red_w/int(reduc_time + 1) + train_w/int(train_time + 1))

def analyse(path, reduction_weight, accuracy_weight, training_weight):
    df_each = []
    for filename in os.listdir(path):
        f = os.path.join(path, filename)
        if os.path.isfile(f) and ".DS" not in f:
            temp = pd.read_csv(f)
            temp = temp[temp.name.apply(lambda x: x != "name")]
            df_each.append((temp, filename))

    included_cols = ["name", "filename", "original_shape", "transformed_shape", "params", "reduction_time", "accuracy", "train_time", "score_series"]

    out_df = pd.DataFrame(columns=included_cols)
    collected_all_df = pd.DataFrame(columns=included_cols)

    for res in df_each:
        res_copy = res[0].copy()
        res_copy["filename"] = res[1]
        score_series = res_copy[res_copy.name != "Nothing"][["reduction_time", "train_time", "accuracy"]].apply(lambda x: score(x["reduction_time"], x["accuracy"], reduction_weight, accuracy_weight, training_weight), axis=1)
        res_copy_score = res_copy.copy()
        res_copy_score["score_series"] = score_series
        out_df = pd.concat([out_df, res_copy_score])
        max_ind = res_copy_score.groupby(by=["original_shape"])["score_series"].idxmax()
        collected = res_copy_score.iloc[max_ind][["name", "filename", "original_shape", "transformed_shape", "params", "reduction_time", "accuracy", "train_time", "score_series"]]
        collected_all_df = pd.concat([collected_all_df, collected])

    all = out_df.groupby(by="filename", group_keys=True).apply(lambda x: x[:])
    best = collected_all_df.groupby(["filename", "name"]).count().sort_values(by="original_shape").groupby(level=0).tail(1).sort_values(by="filename")

    return all, best

In [12]:
reduction_weight = 10
accuracy_weight = 10
training_weight = 10

In [14]:
path = "../output/kmeans/"
kmeans_all, kmeans_best = analyse(path, reduction_weight, accuracy_weight, training_weight)

In [15]:
kmeans_best

,,original_shape,transformed_shape,params,reduction_time,accuracy,train_time,score_series
filename,name,,,,,,,
"1000_0.33_(-100, 100)",extremely sparse JL transform,5,5,5,5,5,5,5
"1000_0.33_(-1000, 1000)",extremely sparse JL transform,5,5,5,5,5,5,5
"1000_0.33_(-500, 500)",extremely sparse JL transform,5,5,5,5,5,5,5
"1000_0.66_(-100, 100)",extremely sparse JL transform,6,6,6,6,6,6,6
"1000_0.66_(-1000, 1000)",extremely sparse JL transform,5,5,5,5,5,5,5
"1000_0.66_(-500, 500)",extremely sparse JL transform,4,4,4,4,4,4,4
"1000_0.99_(-100, 100)",extremely sparse JL transform,4,4,4,4,4,4,4
"1000_0.99_(-1000, 1000)",JL transform,5,5,5,5,5,5,5
"1000_0.99_(-500, 500)",extremely sparse JL transform,3,3,3,3,3,3,3


In [16]:
path = "../output/news/"
news_all, news_best = analyse(path, reduction_weight, accuracy_weight, training_weight)

In [17]:
news_best

,,original_shape,transformed_shape,params,reduction_time,accuracy,train_time,score_series
filename,name,,,,,,,
10,extremely sparse JL transform,1,1,1,1,1,1,1
2,extremely sparse JL transform,1,1,1,1,1,1,1
3,extremely sparse JL transform,1,1,1,1,1,1,1
5,extremely sparse JL transform,1,1,1,1,1,1,1


In [ ]:
d